In [1]:
# Create a new and clear dataset
# The dataset contains only french companies as it is specified in the test statement (Ligne 5 in bold)
# In the new dataset, country_name and city_name are converted to real names that are exploited by the API provided in the test statement (used in the following cells)

# !pip install geopy

from geopy.geocoders import Nominatim
from geopy.distance import geodesic

from geopy.exc import GeocoderTimedOut

import csv
import json

print("HiHi")

# Function to extract city names based on city.json
def cityNames(city_id):
    f = open('city.json', "r", encoding='utf-8')
    city = json.loads(f.read())
    for element in city:
        if element['id'] == city_id:
            city_name = element['name']
            return city_name

# Function to extract country names based on country.json
def countryNames(country_id):
    f = open('country.json', "r", encoding='utf-8')
    city = json.loads(f.read())
    for element in city:
        if element['id'] == country_id:
            country_name = element['name']
            return country_name

# Function to extract longitude and latitude from an address
def longLatAddress(address):
    geolocator = Nominatim(user_agent="https://adresse.data.gouv.fr/api-doc/adresse")
    companyLocation = geolocator.geocode(address)

    if companyLocation is not None:
        companyLocationAddress = companyLocation.address
        companyLocationLatitude = companyLocation.latitude
        companyLocationlongitude = companyLocation.longitude
        # return(companyLocationLatitude, companyLocationlongitude)
    else:
        companyLocationLatitude = None
        companyLocationlongitude = None
    return(companyLocationLatitude, companyLocationlongitude)

# def longLatAddress(address, attempt=1, max_attempts=5):
#     geolocator = Nominatim(user_agent="https://adresse.data.gouv.fr/api-doc/adresse")
#     companyLocation = geolocator.geocode(address)
#     try:
#         if companyLocation is not None:
#             companyLocationAddress = companyLocation.address
#             companyLocationLatitude = companyLocation.latitude
#             companyLocationlongitude = companyLocation.longitude
#             return(companyLocationLatitude, companyLocationlongitude)
#     except GeocoderTimedOut:
#         if companyLocation is None and attempt <= max_attempts:
#             companyLocationLatitude = None
#             companyLocationlongitude = None
#         return(companyLocationLatitude, companyLocationlongitude)

# Function to measure distance between a headquarter and its warehouses
def distanceLocations(headquarter, warehouse):
    distance = geodesic(headquarter, warehouse).kilometers
    if distance < 20:
        clas = "1"
    elif distance < 100 and distance > 20:
        clas = "2"
    elif distance > 100:
        clas = "3"
    return clas

# Function to extract headquarters from the new dataset clearLocations.csv
def extractHeadquarters(addressWarehouse):
    askedFile = open('askedFile.csv', 'w', encoding='utf-8')
    heading = "company_id;country_id;city_id;address;is_headquarter;class\n"
    askedFile.write(heading)
    warehouseLatitude, warehouselongitude = longLatAddress(addressWarehouse)
    lonLatWarehouse = (warehouseLatitude, warehouselongitude)
    for i in range(len(clearLocations)):
        if clearLocations[i][4] == str(1):
            headquarterLatitude, headquarterlongitude = longLatAddress(clearLocations[i][3] + " " + clearLocations[i][2] + " " + clearLocations[i][1])
            if headquarterLatitude is None and headquarterlongitude is None:
                row = "{};{};{};{};{};{}\n".format(clearLocations[i][0], clearLocations[i][1], clearLocations[i][2], clearLocations[i][3], clearLocations[i][4], "AddressIncorrect")
                askedFile.write(row)
            else:
                lonLatHeadquarter = (headquarterLatitude, headquarterlongitude)
                clas = distanceLocations(lonLatHeadquarter, lonLatWarehouse)
                row = "{};{};{};{};{};{}\n".format(clearLocations[i][0], clearLocations[i][1], clearLocations[i][2], clearLocations[i][3], clearLocations[i][4], clas)
                askedFile.write(row)
                askedFile.flush()
        else:
            row = "{};{};{};{};{};{}\n".format(clearLocations[i][0], clearLocations[i][1], clearLocations[i][2], clearLocations[i][3], clearLocations[i][4], "NaN")
            askedFile.write(row)
            askedFile.flush()

HiHi


In [2]:
# Open the provided locations.csv file
with open('locations.csv', encoding='utf-8') as csv_locations:
    csvLocations = csv.reader(csv_locations, delimiter=';')
    locations = list(csvLocations)

# Create the new dataset, named clearLocations.csv
clearLocations = open('clearLocations.csv', "w", encoding='utf-8')
for i in range(len(locations)):
    if locations[i][1] == str(75) and locations[i][3] != 'NULL':
        cityName = cityNames(int(locations[i][2]))
        country_name = countryNames(int(locations[i][1]))
        clearLocations.write("{};{};{};{};{}\n".format(locations[i][0], country_name, cityName, locations[i][3], locations[i][4]))

# Open the created dataset clearLocations.csv
with open('clearLocations.csv') as csv_clearLocations:
    csvClearLocations = csv.reader(csv_clearLocations, delimiter=';')
    clearLocations = list(csvClearLocations)


# addressWarehouse = "16, Place de l'iris, Courbevoie, France"
addressWarehouse = "23, Rue des Jeûneurs, Paris, France"

extractHeadquarters(addressWarehouse)

In [ ]:
!pip install -r requirements.txt